## Timeseries analysis of StatsNZ trade data

In this notebook, I will examine StatsNZ trade data between January 2014 and September 2022. The trade data includes values of NZ Exports and Imports for goods and services per country and product codes. 

The table is available from the Stats NZ trade website: 
<a href="https://statisticsnz.shinyapps.io/trade_dashboard/">Stats NZ Trade Dashboard</a>


#### Importing data and exploring columns

In [1]:
#importing python libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

In [2]:
#reading data into dataframe df

file = '../data/Tradedata.csv'
df = pd.read_csv(file, parse_dates=['Period'], index_col='Period')   

In [3]:
df.head(5)

,Country Code,Country,Account,Category,Code,Flag,Total Value (NZD),Description
Period,,,,,,,,
201406,AF,Afghanistan,Imports,Goods,00,F,131905.0,Total
201406,AF,Afghanistan,Imports,Goods,08,F,107337.0,Fruits and nuts
201406,AF,Afghanistan,Imports,Goods,12,F,316.0,Oil seeds
201406,AF,Afghanistan,Imports,Goods,30,F,21786.0,Pharmaceuticals
201406,AF,Afghanistan,Imports,Goods,42,F,156.0,Leather products


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 413815 entries, 201406 to 202209
Data columns (total 8 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Country Code       413179 non-null  object 
 1   Country            413751 non-null  object 
 2   Account            413815 non-null  object 
 3   Category           413815 non-null  object 
 4   Code               413815 non-null  object 
 5   Flag               413815 non-null  object 
 6   Total Value (NZD)  413815 non-null  float64
 7   Description        413815 non-null  object 
dtypes: float64(1), object(7)
memory usage: 28.4+ MB


The trade table contains 413815 rows and 8 columns. The table does not have null values in the existing columns. 

In [5]:
#creating data frames for exported and imported goods

goods_exports= df[(df["Account"]=='Exports') & (df["Category"]=='Goods') ]
goods_imports= df[(df["Account"]=='Imports') & (df["Category"]=='Goods') ]
goods_exports.tail(2)

,Country Code,Country,Account,Category,Code,Flag,Total Value (NZD),Description
Period,,,,,,,,
202209,ZW,Zimbabwe,Exports,Goods,84,F,17480.0,Mechanical machinery
202209,ZW,Zimbabwe,Exports,Goods,85,F,3187.0,Electrical machinery and equipment


In [6]:
#creating data frames for exported and imported services

services_exports= df[(df["Account"]=='Exports') & (df["Category"]=='Services') ]
services_imports= df[(df["Account"]=='Imports') & (df["Category"]=='Services') ]
services_imports.head(2)

,Country Code,Country,Account,Category,Code,Flag,Total Value (NZD),Description
Period,,,,,,,,
201406,AF,Afghanistan,Imports,Services,A12,F,1205300.0,Total
201406,AF,Afghanistan,Imports,Services,A1204,C,0.0,Travel
